# A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles 
This is the code for the paper entitled "**A Transfer Learning and Optimized CNN Based Intrusion Detection System for Internet of Vehicles**" accepted in IEEE International Conference on Communications (IEEE ICC).  
Authors: Li Yang (lyang339@uwo.ca) and Abdallah Shami (Abdallah.Shami@uwo.ca)  
Organization: The Optimized Computing and Communications (OC2) Lab, ECE Department, Western University

**Notebook 2: CNN Model Development**  
Aims:  
&nbsp; 1): Generate training and test images  
&nbsp; 2): Construct CNN models (a CNN model by own, Xception, VGG16, VGG19, Resnet, Inception, InceptionResnet)  
&nbsp; 3): Tune the hyperparameters of CNN models (hyperparameter optimization)  

## Import libraries

In [1]:
from PIL import Image
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,precision_recall_fscore_support
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import  Xception
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from tensorflow.keras.models import Model,load_model,Sequential
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import plot_model
import datetime
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import sklearn.metrics as metrics
import tensorflow.keras as keras
import tensorflow.keras.callbacks as kcallbacks
import time

## Generate Training and Test Images

In [2]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=128	#could try 128 or 32

train_rootdir = './train_224/'
test_rootdir = './test_224/'

#Normalization
train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_rootdir,
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        test_rootdir,
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')
num_class = train_generator.num_classes

Found 5588 images belonging to 10 classes.
Found 1396 images belonging to 10 classes.


In [3]:
test_labels = []
test_images=[]
for subdir, dirs, files in os.walk(test_rootdir):
    for file in files:
        if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
            continue
        test_labels.append(subdir.split('/')[-1])
        test_images.append(os.path.join(subdir, file))

label=validation_generator.class_indices
label={v: k for k, v in label.items()}

In [4]:
# Prepare the output dir
output_dir = 'output/CNN_based/2-output-{}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
img_dir = os.path.join(output_dir, 'img')
os.makedirs(img_dir)
# Prepare the log file
log_file = open(os.path.join(output_dir, 'classification_report-{}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))), 'w+')

In [5]:
def get_prediction(model, test_images=test_images, label=label, batch_size=BATCHSIZE, verbose='auto'):
    predict=[]
    length=len(test_images)
    for i in range(((length-1)//batch_size)+1):
        inputimg=test_images[batch_size*i:batch_size*(i+1)]
        test_batch=[]
        for path in inputimg:
            thisimg=np.array(Image.open(path))/255
            test_batch.append(thisimg)
        # thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
        #print(thisimg)
        # test_shape=(1,)+thisimg.shape
        # thisimg=thisimg.reshape(test_shape)
        model_batch=model.predict(np.array(test_batch), verbose=verbose) #use master model to process the input image
        #generate result by model 1
        # prob=model_batch[0,np.argmax(model_batch,axis=1)[0]]
        predict_batch=list(np.argmax(model_batch,axis=1))
        predict_batch=[label[con] for con in predict_batch]
        predict.extend(predict_batch)
    return predict

### Define the image plotting functions

In [6]:
#plot the figures
#when extra_data enabled please put this callback before early_stopping in case of any problem
class LossHistory(keras.callbacks.Callback):
    def __init__(self, need_extra_data:bool=True, test_images=test_images, test_labels=test_labels, label=label):
        # Enable the recording of precision, recall, f1-score, prediction time (only epoch-wise)
        self.extra_data = need_extra_data
        if need_extra_data:
            self.test_images = test_images
            self.test_labels = test_labels
            self.label = label
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_accuracy = {'batch':[], 'epoch':[]}
        # need extra data --> reset recording list
        # These matrics only make sense over the entire epoch
        if self.extra_data:
            self.precision = []
            self.recall = []
            self.f1_score = []
            self.predict_time = []
            # Record prediction -> for generating report
            self.prediction = []
    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_accuracy['batch'].append(logs.get('val_accuracy'))
    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_accuracy['epoch'].append(logs.get('val_accuracy'))
        # need extra data --> calculate and record
        if self.extra_data:
            # Get prediciton
            temp = self.model.stop_training
            start_time = time.time()
            y_pred = get_prediction(model=self.model, test_images=self.test_images, label=self.label, verbose='auto')
            end_time = time.time()
            self.model.stop_training = temp
            # Calculate extra data
            precision,recall,fscore,_= precision_recall_fscore_support(self.test_labels, y_pred, average='weighted', zero_division=0)
            # Record
            self.precision.append(precision)
            self.recall.append(recall)
            self.f1_score.append(fscore)
            self.predict_time.append((end_time-start_time)/len(y_pred))
            # Save prediction
            self.prediction.append(y_pred)
    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # acc
            plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
            # loss
            plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
            # val_acc
            plt.plot(iters, self.val_accuracy[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        else:
            plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()
    def get_best(self, target_type:str='epoch', need_extra_data:bool=True):
        # Get the index of the best record
        max_index = self.accuracy[target_type].index(max(self.accuracy[target_type]))
        # Return the accuracy, loss, val_acc, val_loss of the best record
        temp={
            'accuracy': self.accuracy[target_type][max_index], 
            'loss': self.losses[target_type][max_index], 
            'val_accuracy': self.val_accuracy[target_type][max_index], 
            'val_loss': self.val_loss[target_type][max_index]
            }
        # Add extra data if needed and available
        if self.extra_data and need_extra_data and target_type=='epoch':
            temp['precision']=self.precision[max_index]
            temp['recall']=self.recall[max_index]
            temp['f1-score']=self.f1_score[max_index]
            temp['predict_time_per_image']=self.predict_time[max_index]
        return temp
    def generate_report(self, name:str, img_dir=img_dir, log_file=log_file, epoch='best', figsize=(18,14), log_classification_report:bool=True, save_img:bool=True, print_classifiaction_report:bool=True, display_confusion_matrix:bool=True):
        if not self.extra_data:
            raise Exception('Prediction record unavailable')
        # Get prediction
        if epoch=='best': epoch=self.accuracy['epoch'].index(max(self.accuracy['epoch']))
        elif epoch=='worst': epoch=self.accuracy['epoch'].index(min(self.accuracy['epoch']))
        else: epoch-=1
        y_pred = self.prediction[epoch]
        # Generate classification report
        report_str = classification_report(self.test_labels,y_pred,zero_division=0)
        if log_classification_report and log_file.writable(): log_file.write('******{}******\n'.format(name)+report_str+'\n')
        if print_classifiaction_report: print(report_str)
        # Generate confusion matrix
        cm=confusion_matrix(self.test_labels,y_pred)
        f,ax=plt.subplots(figsize=figsize)
        sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
        ax.set_xticklabels(self.label.values())
        ax.set_yticklabels(self.label.values())
        plt.xlabel("y_pred")
        plt.ylabel("y_true")
        if save_img: plt.savefig(os.path.join(img_dir, '{}.pdf'.format(name)))
        if display_confusion_matrix: plt.show()


In [7]:
history_this = LossHistory(need_extra_data=True)
history_hpo = LossHistory(need_extra_data=False)

### Define the processing time measurement functions

In [8]:
# Measure the running time of model training
class TimeMeasurement(keras.callbacks.Callback):
    def __init__(self):
        self.start_time=None
        self.stop_time=None
    # Start timing when trainning starts
    def on_train_begin(self, logs=None):
        self.start_time=time.time()
        self.stop_time=None
    # Stop timing when trainning ends
    def on_train_end(self, logs=None):
        self.stop_time=time.time()
        if self.start_time is None: print("Time Measuring Failed")
    # Get processing time
    def get_processing_time(self):
        if (self.start_time is None or self.stop_time is None): raise Exception("Wrong Time Measurement")
        else: return self.stop_time-self.start_time

In [9]:
timer = TimeMeasurement()

### Define the output sheet

In [10]:
class output_sheet:
    def __init__(self, columns:list=['accuracy', 'loss', 'val_accuracy', 'val_loss', 'precision', 'recall', 'f1-score', 'hpo_time', 'train_time', 'predict_time_per_image']):
        self.output_df = pd.DataFrame(columns=columns)
        # self.output_index = list()
    def add(self, item:str, **values:dict):
        # self.output_df = self.output_df.append(values, ignore_index=True)
        temp = pd.DataFrame(values, columns=self.output_df.columns.to_list(), index=[item])
        self.output_df = pd.concat([self.output_df, temp], axis=0)
        # self.output_index.append(item)
    # def apply_index(self):
    #     self.output_df.index = self.output_index
    def to_excel(self, path=None):
        if path is None: path=os.path.join(output_dir, '2-result-{}.xlsx'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S')))
        # self.apply_index()
        self.output_df.to_excel(path)

In [11]:
output = output_sheet(columns=['accuracy', 'loss', 'val_accuracy', 'val_loss', 'precision', 'recall', 'f1-score', 'hpo_time', 'train_time', 'predict_time_per_image'])

# Construct CNN models

### Model 1: a CNN model by own (baseline)

In [12]:
def cnn_by_own(input_shape,num_class,epochs,savepath='./model_own.h5',history=history_this,timer=timer):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(num_class,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_accuracy', patience=2, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
    hist=model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )

In [13]:
cnn_by_own(input_shape=INPUT_SIZE,num_class=num_class,epochs=20)
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

d:\Software\anaconda3\envs\IDSml_cnn_2\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d/Relu (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\1467547391.py:20) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1241]

Function call stack:
train_function


Validation accuracy of a CNN by own: 99.884%

In [12]:
output.add('model_own', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('model_own_original')

### Model 2: Xception

In [14]:
def xception( num_class, epochs,savepath='./xception.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:131]:		#could be tuned to be 50, 100, or 131
        layer.trainable = False
    for layer in model_fine_tune.layers[131:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='xception')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=3, verbose=1, mode='auto')	#patience could be tuned by 2 and 3
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )


In [15]:
#default only 50, tf36cnn 99
xception(num_class=num_class,epochs=20)
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

d:\Software\anaconda3\envs\IDSml_cnn_2\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,128,109,109] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node xception/block2_sepconv1/separable_conv2d (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\2818243488.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_8336]

Function call stack:
train_function


Validation accuracy of Xception: 100.0%

In [16]:
output.add('Xception', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Xception_original')

### Model 3: VGG16

In [16]:
def vgg16( num_class, epochs,savepath='./VGG16.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:15]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[15:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output) #GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )


In [17]:
vgg16(num_class=num_class,epochs=20)	#tf36cnn
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node vgg/block1_conv1/Relu (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\738096641.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_9798]

Function call stack:
train_function


Validation accuracy of VGG16: 100.0%

In [12]:
output.add('VGG16', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG16_original')

### Model 4: VGG19

In [18]:
def vgg19( num_class, epochs,savepath='./VGG19.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:19]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[19:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )


In [19]:
vgg19(num_class=num_class,epochs=20)	#binary classificaiton
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

80142336/80134624 [==============================] - 22s 0us/step
Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node vgg/block1_conv1/Relu (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\1240161684.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_11381]

Function call stack:
train_function


Validation accuracy of VGG19: 100.0%

In [ ]:
output.add('VGG19', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG19_original')

### Model 5: ResNet

In [20]:
def resnet( num_class, epochs,savepath='./resnet.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:120]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[120:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )

In [21]:
resnet(num_class=num_class,epochs=20)	#binary classificaiton
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

94773248/94765736 [==============================] - 39s 0us/step
Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[8192,114,114] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resnet/pool1_pad/Pad (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\1254716126.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_21151]

Function call stack:
train_function


Validation accuracy of Resnet: 98.652%

In [ ]:
output.add('Resnet', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('ResNet_original')

### Model 6: Inception

In [22]:
def inception( num_class, epochs,savepath='./inception.h5',history=history_this,input_shape=INPUT_SIZE, timer=timer):
    model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:35]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[35:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )

In [23]:
inception(num_class=num_class,epochs=20)	#binary classificaiton
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

87916544/87910968 [==============================] - 17s 0us/step
Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,109,109] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resnet/conv2d_13/Conv2D (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\885599727.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_36235]

Function call stack:
train_function


Validation accuracy of Inception: 100.0%

In [ ]:
output.add('Inception', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Inception_original')

### Model 7: InceptionResnet

In [24]:
def inceptionresnet( num_class, epochs,savepath='./inceptionresnet.h5',history=history_this,input_shape=INPUT_SIZE, timer=timer):
    model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:500]:	#the number of frozen layers for transfer learning, have tuned from 400-550
        layer.trainable = False
    for layer in model_fine_tune.layers[500:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(0.5)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=2, verbose=1, mode='auto')	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )

In [25]:
inceptionresnet(num_class=num_class,epochs=20)	# 5-class classificaiton
history_this.loss_plot('epoch')
history_this.loss_plot('batch')
plt.show()

219062272/219055592 [==============================] - 27s 0us/step
Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[128,32,111,111] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resnet/batch_normalization_98/FusedBatchNormV3 (defined at C:\Users\xht30\AppData\Local\Temp\ipykernel_51316\1805849801.py:23) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_68493]

Function call stack:
train_function


Validation accuracy of InceptionResnet: 99.993%

In [ ]:
output.add('InceptionResnet', train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('InceptionResnet_original')

# Hyperparameter Optimization 
Use VGG16 as an example.  

Tuned hyperparameters of CNN: 
1. The number of frozen layers
2. The number of epochs
3. Early stop patience
4. Learning rate
5. Dropout rate

Hyperparameter optimization methods:
1. Random search
2. Bayesian optimization - Tree Parzen Estimator(BO-TPE)

In [14]:
def prediction(model, test_labels=test_labels, test_images=test_images, label=label):
#read images from validation folder
    # test_labels = []
    # test_images=[]
    # for subdir, dirs, files in os.walk(rootdir):
    #     for file in files:
    #         if not (file.endswith(".jpeg"))|(file.endswith(".jpg"))|(file.endswith(".png")):
    #             continue
    #         test_labels.append(subdir.split('/')[-1])
    #         test_images.append(os.path.join(subdir, file))

    # label=validation_generator.class_indices
    # label={v: k for k, v in label.items()}
    # predict=[]
    # length=len(test_images)
    # for i in range(length):
    #     inputimg=test_images[i]
    #     test_batch=[]
    #     thisimg=np.array(Image.open(inputimg))/255 #read all the images in validation set
    #     #print(thisimg)
    #     test_shape=(1,)+thisimg.shape
    #     thisimg=thisimg.reshape(test_shape)
    #     model_batch=model.predict(thisimg) #use master model to process the input image
    #     #generate result by model 1
    #     prob=model_batch[0,np.argmax(model_batch,axis=1)[0]]
    #     res=label[np.argmax(model_batch,axis=1)[0]]
    #     predict.append(res)
    acc=accuracy_score(test_labels, get_prediction(model=model, test_images=test_images, label=label))
    return acc

## Model by Own

In [ ]:
def cnn_by_own(num_class,input_shape=INPUT_SIZE,epochs=20,patience=2, dropout_rate=0.5,verbose=0,savepath='./model_own.h5',history=history_this,timer=timer):
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(num_class,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    #train model
    earlyStopping=kcallbacks.EarlyStopping(monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)
    saveBestModel = kcallbacks.ModelCheckpoint(filepath=savepath, monitor='val_accuracy', verbose=verbose, save_best_only=True, mode='auto')
    hist=model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        callbacks=[history, timer, earlyStopping, saveBestModel],
        verbose=verbose
    )
    return model

In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = cnn_by_own(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
cnn_by_own(input_shape=INPUT_SIZE, num_class=num_class, verbose=1, **params)

In [ ]:
output.add('model_own (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('model_own_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
space = {
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
cnn_by_own(input_shape=INPUT_SIZE, num_class=num_class, verbose=1, **params)

In [ ]:
output.add('model_own (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('model_own_Random_Search')

## Xception

In [ ]:
def xception( num_class,epochs=20,frozen=15,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./xception.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = Xception(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:		#could be tuned to be 50, 100, or 131
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='xception')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#patience could be tuned by 2 and 3
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        verbose=verbose,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )
    return model


In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = xception(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
available_frozen = [50, 100, 131]
space = {
    'frozen': hp.choice('frozen', available_frozen),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': available_frozen[int(best['frozen'])],
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
xception(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('Xception (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Xception_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
available_frozen = [50, 100, 131]
space = {
    'frozen': hp.choice('frozen', available_frozen),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': available_frozen[int(best['frozen'])],
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
xception(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('Xception (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Xception_Random_Search')

## VGG16

In [13]:
def vgg16(num_class,epochs=20,frozen=15,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0, savepath='./VGG16.h5',history=history_this,timer=timer,input_shape=INPUT_SIZE):
    model_fine_tune = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[history, timer, earlyStopping, saveBestModel],
        verbose = verbose
    )
    return model


In [15]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = vgg16(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [16]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


accuracy:0.9842406876790831                           
accuracy:0.9598853868194842                                                         
accuracy:0.9756446991404012                                                         
accuracy:0.9713467048710601                                                       
 40%|████      | 4/10 [20:47<29:57, 299.59s/trial, best loss: -0.9842406876790831]WARNING:tensorflow:Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


accuracy:0.9684813753581661                                                       
accuracy:0.9570200573065902                                                       
 60%|██████    | 6/10 [31:07<20:06, 301.50s/trial, best loss: -0.9842406876790831]WARNING:tensorflow:Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


accuracy:0.9749283667621776                                                       
accuracy:0.9777936962750716                                                       
accuracy:0.9799426934097422                                                       
accuracy:0.9634670487106017                                                       
100%|██████████| 10/10 [51:03<00:00, 306.39s/trial, best loss: -0.9842406876790831]
Hyperopt estimated optimum {'dropout_rate': 0.6000000000000001, 'epochs': 15.0, 'frozen': 16.0, 'lr': 0.002, 'patience': 3.0}
Time: 3063.9442534446716


In [17]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
vgg16(num_class=num_class, verbose=1, **params)

Epoch 1/15
11/44 [======>.......................] - ETA: 20s - loss: 0.2106 - acc: 0.9384
Epoch 00001: val_acc improved from -inf to 0.93840, saving model to ./VGG16.h5
44/44 [==============================] - 36s 825ms/step - loss: 0.8529 - acc: 0.8219 - val_loss: 0.2106 - val_acc: 0.9384
Epoch 2/15
11/44 [======>.......................] - ETA: 19s - loss: 0.1864 - acc: 0.9384
Epoch 00002: val_acc did not improve from 0.93840
44/44 [==============================] - 26s 601ms/step - loss: 0.1814 - acc: 0.9370 - val_loss: 0.1864 - val_acc: 0.9384
Epoch 3/15
11/44 [======>.......................] - ETA: 21s - loss: 0.1434 - acc: 0.9477
Epoch 00003: val_acc improved from 0.93840 to 0.94771, saving model to ./VGG16.h5
44/44 [==============================] - 28s 631ms/step - loss: 0.1554 - acc: 0.9472 - val_loss: 0.1434 - val_acc: 0.9477
Epoch 4/15
11/44 [======>.......................] - ETA: 21s - loss: 0.1229 - acc: 0.9506
Epoch 00004: val_acc improved from 0.94771 to 0.95057, saving m

In [18]:
output.add('VGG16 (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG16_BO-TPE')

### Random Search

In [20]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

accuracy:0.9684813753581661                           
accuracy:0.9684813753581661                                                       
accuracy:0.9756446991404012                                                       
accuracy:0.9670487106017192                                                       
accuracy:0.9806590257879656                                                       
accuracy:0.9813753581661891                                                       
accuracy:0.9820916905444126                                                       
accuracy:0.9792263610315186                                                       
accuracy:0.9584527220630372                                                       
accuracy:0.9777936962750716                                                       
100%|██████████| 10/10 [54:06<00:00, 324.61s/trial, best loss: -0.9820916905444126]
Hyperopt estimated optimum {'dropout_rate': 0.4, 'epochs': 15.0, 'frozen': 18.0, 'lr': 0.004, 'patience': 4.0}
Tim

In [21]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
vgg16(num_class=num_class, verbose=1, **params)

Epoch 1/15
11/44 [======>.......................] - ETA: 19s - loss: 0.2843 - acc: 0.9463
Epoch 00001: val_acc improved from -inf to 0.94628, saving model to ./VGG16.h5
44/44 [==============================] - 41s 924ms/step - loss: 0.5985 - acc: 0.8416 - val_loss: 0.2843 - val_acc: 0.9463
Epoch 2/15
11/44 [======>.......................] - ETA: 19s - loss: 0.1756 - acc: 0.9542
Epoch 00002: val_acc improved from 0.94628 to 0.95415, saving model to ./VGG16.h5
44/44 [==============================] - 27s 613ms/step - loss: 0.2357 - acc: 0.9418 - val_loss: 0.1756 - val_acc: 0.9542
Epoch 3/15
11/44 [======>.......................] - ETA: 19s - loss: 0.1313 - acc: 0.9620
Epoch 00003: val_acc improved from 0.95415 to 0.96203, saving model to ./VGG16.h5
44/44 [==============================] - 27s 609ms/step - loss: 0.1636 - acc: 0.9588 - val_loss: 0.1313 - val_acc: 0.9620
Epoch 4/15
11/44 [======>.......................] - ETA: 19s - loss: 0.1156 - acc: 0.9642
Epoch 00004: val_acc improved f

In [22]:
output.add('VGG16 (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG16_Random_Search')

## VGG19

In [ ]:
def vgg19( num_class,epochs=20,frozen=15,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./VGG19.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = VGG19(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 5-18
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='vgg')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])	#set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        verbose=verbose,
        #use_multiprocessing=True, 
        #workers=2,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )
    return model


In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = vgg19(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
vgg19(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('VGG19 (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG19_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 15, 18, 1),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
vgg19(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('VGG19 (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('VGG19_Random_Search')

## ResNet

In [ ]:
def resnet( num_class, epochs=20,frozen=120,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./resnet.h5',history=history_this,input_shape=INPUT_SIZE,timer=timer):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        verbose=verbose,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )
    return model

In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = resnet(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 50, 150, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
resnet(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('ResNet (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('ResNet_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 50, 150, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
resnet(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('ResNet (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('ResNet_Random_Search')

## Inception

In [ ]:
def inception( num_class, epochs=20,frozen=120,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./inception.h5',history=history_this,input_shape=INPUT_SIZE, timer=timer):
    model_fine_tune = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 50-150
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        verbose=verbose,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )
    return model

In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = inception(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 50, 150, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
inception(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('Inception (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Inception_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 50, 150, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
inception(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('Inception (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('Inception_Random_Search')

## InceptionResnet

In [ ]:
def inceptionresnet( num_class, epochs=20,frozen=120,learning_rate=0.001,patience=2, dropout_rate=0.5,verbose=0,savepath='./inceptionresnet.h5',history=history_this,input_shape=INPUT_SIZE, timer=timer):
    model_fine_tune = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:	#the number of frozen layers for transfer learning, have tuned from 400-550
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:	#the number of trainable layers for transfer learning
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model=Dense(units=256,activation='relu')(model)
    model=Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)	#tuned learning rate to be 0.001
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy']) #set the loss function to be binary crossentropy
    #train model
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=verbose, mode='auto', restore_best_weights=True)	#set early stop patience to save training time
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=verbose,
        save_best_only=True,
        mode='auto')
    hist = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        #use_multiprocessing=True, 
        verbose=verbose,
        callbacks=[history, timer, earlyStopping, saveBestModel],
    )
    return model

In [ ]:
#define the objective function to be optimized
import time
from hyperopt import hp, fmin, tpe, rand, STATUS_OK, Trials
import matplotlib.pyplot as plt
import statistics 

def objective(params, num_class=num_class, history=history_hpo):
    
    params = {
        'frozen': int(params['frozen']),
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'learning_rate': abs(float(params['learning_rate'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    # frozen=params['frozen']
    # epochs=params['epochs']
    # patience=params['patience']
    # learning_rate=params['learning_rate']
    # dropout_rate=params['dropout_rate']

    # vgg16(num_class=num_class, frozen=frozen,epochs=epochs,patience=patience, learning_rate=learning_rate, dropout_rate=dropout_rate)

    model = inceptionresnet(num_class=num_class, history=history, **params)

    acc=prediction(model=model)

    print('accuracy:%s'%acc)
    return {'loss': -acc, 'status': STATUS_OK }
    

### BO-TPE

In [ ]:
#Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 400, 500, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
inceptionresnet(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('InceptionResnet (BO-TPE)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('InceptionResnet_BO-TPE')

### Random Search

In [ ]:
#Hyperparameter optimization by Random search
space = {
    'frozen': hp.quniform('frozen', 400, 500, 10),
    'epochs': hp.quniform('epochs', 5, 21, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'learning_rate': hp.quniform('learning_rate', 0.001, 0.006, 0.001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

t1=time.time()
best = fmin(fn=objective,
            space=space,
            algo=rand.suggest,
            max_evals=10)
print("Hyperopt estimated optimum {}".format(best))
t2=time.time()
print("Time: "+str(t2-t1))

In [ ]:
# Retrain the model by using the best hyperparameter values to obtain the best model
params = {
        'frozen': int(best['frozen']),
        'epochs': int(best['epochs']),
        'patience': int(best['patience']),
        'learning_rate': abs(float(best['learning_rate'])),
        'dropout_rate': abs(float(best['dropout_rate'])),
    }
inceptionresnet(num_class=num_class, verbose=1, **params)

In [ ]:
output.add('InceptionResnet (Random Search)', hpo_time=t2-t1, train_time=timer.get_processing_time(), **history_this.get_best())
history_this.generate_report('InceptionResnet_Random_Search')

# Save Result

In [24]:
output.to_excel()
log_file.close()

In [ ]:
# # Online GPU renting platform specification
# # WeChat Message
# import requests
# resp = requests.get(
#     "https://www.autodl.com/api/v1/wechat/message/push?token={token}&title={title}&name={name}&content={content}".format(
#         token="",
#         title="From AutoDL",
#         name="UNSW-NB15 CNN",
#         content="Training Complete")
# )
# print(resp.content.decode())
# # Shutdown
# !shutdown